# Leavitt experiment using LLMs as nodes

### Setup

In [74]:
from __future__ import annotations   # needed for Python < 3.14 for type hint of Node class

import networkx as nx
from dataclasses import dataclass, field
from typing import Dict, List, Set, Tuple, Optional, Literal, Type
import matplotlib.pyplot as plt
import random
import json

from openai import OpenAI

from dotenv import load_dotenv
_ = load_dotenv()

#### Define networks and allowed nodes

In [3]:
NODES = ('A','B','C', 'D', 'E')

SYMBOLS: List[str] = [  '\u25A1', # square
                        '\u25B7', # right triangle
                        '\u25C1', # left triangle
                        '\u25C7', # diamond
                        '\u25CB', # circle
                        '\u2606', # star     
                     ]

NETWORKS: Dict[str, Dict[str, List[str]]] = {
    "chain": {
        "A": ["B"],
        "B": ["A", "C"],
        "C": ["B", "D"],
        "D": ["C", "E"],
        "E": ["D"],
    },
    "circle": {
        "A": ["B", "E"],
        "B": ["A", "C"],
        "C": ["B", "D"],
        "D": ["C", "E"],
        "E": ["D", "A"],
    },
    "wheel": {  
        "A": ["C"],
        "B": ["C"],
        "C": ["A", "B", "D", "E"],
        "D": ["C"],
        "E": ["C"],
    },
    "Y": {  
        "A": ["C"],
        "B": ["C"],
        "C": ["A", "B", "D"],
        "D": ["C", "E"],
        "E": ["D"],
    },
}

##### Function to create trial data

* Using the original symbol patterns with one common symbol across all participants

In [4]:
def generate_trial(symbols: List[str],
                   nodes: List[str]):
    '''
    Generate trial data for N nodes from a list of S>N (usually N+1) symbols.
    Constraints: only one symbol from list occurs in ALL node data

    Returns:
        common_symbol: str
        trial_data: Dict[str, List[str]]
    '''

    common_symbol = random.choice(symbols)
    others = [sym for sym in symbols if sym!=common_symbol]
    random.shuffle(others)
    trial_data = {}
    for node, missing in zip(nodes, others):
        trial_data[node] = [sym for sym in symbols if sym!=missing]
        random.shuffle(trial_data[node])
    
    return common_symbol, trial_data



In [5]:
generate_trial(SYMBOLS, NODES)

('☆',
 {'A': ['◇', '☆', '◁', '□', '○'],
  'B': ['☆', '□', '▷', '◁', '◇'],
  'C': ['☆', '○', '◁', '▷', '□'],
  'D': ['□', '☆', '○', '▷', '◇'],
  'E': ['○', '◇', '☆', '◁', '▷']})

### Prompts

In [6]:

SYSTEM_PROMPT =  """
        You are one participant (node) in a 5-person communication network.
        Each of you has been given a card with 5 symbols on it.
        You are trying to find the SINGLE symbol that appears on all 5 cards.
        You ONLY know your own card symbols and messages you receive.
        You may send written messages ONLY to your listed neighbors.
        
        Rules:
        - Do not invent symbols you haven't seen.
        - Prefer sending compact, useful info (e.g., your card or your current candidate set).
        - When you are confident the common symbol is uniquely determined, set current_guess.
        - Output MUST be valid JSON with keys: current_guess, outgoing_messages, notes.
        - outgoing_messages is a list of {to, content} where to is a neighbor.
        """

TURN_PROMPT = """
        You current knowledge is {current_knowledge}

        You can ONLY send messages to NODES {neighbor_str}

        What do you want to do now?
        """

#### Node class 

In [160]:
class Node:

    def __init__(self, 
                 id: str, 
                 network: str,
                 card: str
                ):

        if id not in NODES:
            raise Exception(f'Node id must be one of {NODES}')
        
        self.id = id

        if not network in NETWORKS.keys():
            raise Exception(f'Network must be one of: {NETWORKS.keys()}')
        
        self.network = network

        self.neighbors = NETWORKS[network][id]
        self.current_guess = None
        self.notes = []
        self.initial_data = card
        self.inbox = []
        self.outbox = []

    def __repr__(self):
        return f"""Node(id='{self.id}', network={self.network}, 
                   neighbors={self.neighbors},
                   initial_data={self.initial_data})
                   inbox={self.inbox},
                   outbox={self.outbox},
                   notes={self.notes}
                   """


    def send_message(self, recipient: Node, content: str):
        """
        Send a message `content` to a neighbor checking it is allowed
        Returns:
            status
        """

        recipient_id = recipient.id

        if recipient_id not in self.neighbors:
            raise Exception(f'Trying to send a message to an non-neighbor')

        try:
            recipient.receive_message(self.id, content)
            self.outbox.append({'to': recipient_id,
                                'message': content})
            return f"Message '{content[:10]}...' sent to Node {recipient_id}'"
        except Exception as e:
            print(e)
            return False


    def receive_message(self, sender: str, content: str):
        """
        Receive a message from a sender and add to inbox
        """

        self.inbox.append({'from': sender, 'message': content})


    def take_turn(self):
        """
        Evaluate current knowledge and decide what action to take
        - send message to a neighbor
        - make a guess
        """

        #if self.current_guess:
        #    return f"Node {self.id} has guessed {self.current_guess}"
        
        current_inbox = '\n'.join([ f"{message['from']} said {message['message']}" for message in self.inbox]) if self.inbox else "No interactions"

        notes = '\n'.join(self.notes)

        
        current_knowledge = f"""
        Your initial data is: { self.initial_data }
        You have learned the following from your interactions:
        Your observations:
        {notes}
        Messages you have received:
        {current_inbox}
        """

        messages = [
            {'role': 'system',
             'content': SYSTEM_PROMPT },

            {'role': 'user',
             'content': TURN_PROMPT.format(current_knowledge=current_knowledge, 
                                           neighbor_str = ' and '.join(self.neighbors)) },
        ]

        # TODO - give each node instance a specific LLM
        response = LLM.chat.completions.create(
            model="chatgpt-4o-latest",
            messages=messages,
            response_format={"type": "json_object"}
        )

        resp = json.loads(response.choices[0].message.content)

        if resp['current_guess']:
            self.current_guess = resp['current_guess']
        
        self.notes.append(resp['notes'])
        
        # send outgoing messages
        for message in resp['outgoing_messages']:
            self.send_message(nodes[message['to']], message['content'])
        
        return resp

### Testing

In [161]:
LLM = OpenAI()

In [171]:
network_type = 'circle'

common_symbol, trial_data = generate_trial(SYMBOLS, NODES)

nodes = {node: Node(id=node, 
              network=network_type, 
              card = ' '.join(trial_data[node]))
              for node in NETWORKS[network_type]
        }

In [172]:
nodes

{'A': Node(id='A', network=circle, 
                    neighbors=['B', 'E'],
                    initial_data=○ ☆ ◁ ◇ ▷)
                    inbox=[],
                    outbox=[],
                    notes=[]
                    ,
 'B': Node(id='B', network=circle, 
                    neighbors=['A', 'C'],
                    initial_data=○ ▷ ◇ □ ☆)
                    inbox=[],
                    outbox=[],
                    notes=[]
                    ,
 'C': Node(id='C', network=circle, 
                    neighbors=['B', 'D'],
                    initial_data=☆ □ ○ ◇ ◁)
                    inbox=[],
                    outbox=[],
                    notes=[]
                    ,
 'D': Node(id='D', network=circle, 
                    neighbors=['C', 'E'],
                    initial_data=□ ▷ ◁ ○ ☆)
                    inbox=[],
                    outbox=[],
                    notes=[]
                    ,
 'E': Node(id='E', network=circle, 
                    neighbor

In [173]:
common_symbol

'○'

In [174]:
guesses = dict(zip(NODES, [None]*len(NODES)))

round = 1 
while None in guesses.values():
    print(f'\nROUND {round}\n===========')
    for nid, node in nodes.items():
        node.take_turn()
        print(f"{nid} current guess {node.current_guess}")
        if node.current_guess:
            guesses[nid]=node.current_guess

    round+=1


ROUND 1
A current guess None
B current guess None
C current guess None
D current guess None
E current guess None

ROUND 2
A current guess ○
B current guess None
C current guess None
D current guess ○
E current guess None

ROUND 3
A current guess ○
B current guess ○
C current guess ○
D current guess ○
E current guess ○


In [175]:
nodes

{'A': Node(id='A', network=circle, 
                    neighbors=['B', 'E'],
                    initial_data=○ ☆ ◁ ◇ ▷)
                    inbox=[{'from': 'B', 'message': 'Thanks! My symbols: ○ ▷ ◇ □ ☆'}, {'from': 'E', 'message': "Here's my card: ▷ ○ ◁ ◇ □"}, {'from': 'E', 'message': 'I have cards from you, me, and D. The overlap so far includes ▷, ○, and ◁. Do you know anything from B or C?'}, {'from': 'E', 'message': 'Thanks for the update. Based on cards from me, A, D, and B (via you), ○ is the only symbol common to all. I agree with your guess.'}],
                    outbox=[{'to': 'B', 'message': 'My symbols: ○ ☆ ◁ ◇ ▷'}, {'to': 'E', 'message': 'My symbols: ○ ☆ ◁ ◇ ▷'}, {'to': 'E', 'message': "Yes, I got B's card. Symbols are: ○ ▷ ◇ □ ☆. From me, B, and you, only ○ appears in all three. I think that's the common symbol."}, {'to': 'B', 'message': 'E also shared their card with me: ▷ ○ ◁ ◇ □. From me, you, and E, only ○ is shared by all. I think ○ is the common symbol.'}],
     